In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn import metrics
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import KFold

import random, os, json
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, GRU, Dropout, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

import methods

Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


## Functions

In [2]:
def reset_keras(seed=42):
    """Function to ensure that results from Keras models
    are consistent and reproducible across different runs"""
    
    K.clear_session()
    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    os.environ['PYTHONHASHSEED']=str(seed)
    # 2. Set `python` built-in pseudo-random generator at a fixed value
    random.seed(seed)
    # 3. Set `numpy` pseudo-random generator at a fixed value
    np.random.seed(seed)
    # 4. Set `tensorflow` pseudo-random generator at a fixed value
    tf.random.set_seed(seed)


class GRUModel:
    """
    GRUModel class builds and trains a Gated Recurrent Unit (GRU) model
    with specified layers and hyperparameters.
    
    Attributes:
    -----------
    hyperparameters : dict
        A dictionary containing key hyperparameters for model building and training.
        
    Methods:
    --------
    build_model(lr_sch):
        Builds the GRU model with the specified learning rate scheduler.
    train(x_train, y_train, epochs, batch_size, validation_data):
        Trains the built model with the provided training and validation data.
    """
    
    def __init__(self, hyperparameters):
        """
        Initializes the GRUModel with hyperparameters.
        
        Parameters:
        -----------
        hyperparameters : dict
            A dictionary containing key hyperparameters for model building and training.
        """
        self.hyperparameters = hyperparameters
        
    def build_model(self, lr_sch):
        """
        Builds the GRU model with specified learning rate scheduler.
        
        Parameters:
        -----------
        lr_sch : float
            Learning rate for the optimizer during training.
            
        Returns:
        --------
        model : tf.keras.Model
            The compiled GRU model.
        """
        # Define input layer with dynamic shape and masking
        dynamic_input = tf.keras.layers.Input(shape=(self.hyperparameters["timeStep"], self.hyperparameters["layers"][0]))
        masked = tf.keras.layers.Masking(mask_value=self.hyperparameters['maskValue'])(dynamic_input)
        
        weighted, hadamard_scores = methods.hadamard_attention(masked, self.hyperparameters["layers"][0], self.hyperparameters['dropout'])
            
        # Define GRU layer with specified parameters
        gru_encoder = tf.keras.layers.GRU(
            self.hyperparameters['layers'][1],
            dropout=self.hyperparameters['dropout'],
            return_sequences=True,
            activation='tanh',
            use_bias=True
        )(masked)

        # Define output layer with sigmoid activation function
        output = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1, use_bias=False, activation="sigmoid"))(gru_encoder)
        
        # Compile the model with Adam optimizer and custom loss function
        model = tf.keras.Model(dynamic_input, [output])
        my_optimizer = tf.keras.optimizers.Adam(learning_rate=lr_sch)
        model.compile(loss="binary_crossentropy",
                      optimizer=my_optimizer,
                      metrics=['accuracy', 'AUC']
                     )
        
        return model
        
    def train(self, x_train, y_train, x_val, y_val):
        """
        Trains the built model with provided training and validation data.
        
        Parameters:
        -----------
        x_train : numpy array
            Input training data.
        y_train : numpy array
            Target training data.
        epochs : int
            Number of training epochs.
        batch_size : int
            Batch size for training.
        validation_data : tuple
            Tuple containing input and target validation data.
        
        Returns:
        --------
        history : tf.keras.callbacks.History
            A record of training loss values and metrics values at successive epochs.
        model : tf.keras.Model
            The trained GRU model.
        """
        
        model = self.build_model(lr_sch=self.hyperparameters['lr_scheduler'])
        earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      min_delta=self.hyperparameters["mindelta"],
                                                      patience=self.hyperparameters["patience"],
                                                      restore_best_weights=True,
                                                      mode="min")
                    
        history = model.fit(x_train, y_train,
                            validation_data=(x_val, y_val),
                            callbacks=[earlystopping],
                            batch_size=self.hyperparameters['batch_size'], 
                            epochs=self.hyperparameters['epochs'],
                            verbose=self.hyperparameters['verbose'])

        return history, model
    
    
def myCVGrid(hyperparameters, seed, X_train, y_train):
    """Grid Search. Calculate metricDev based on the evaluation. Compares the metricDev with the current bestMetricDev. 
       If better, updates bestMetricDev and stores those hyperparameters in bestHyperparameters.
       
       Returns:
          - bestHyperparameters (dict)
          - X_train, X_val, y_train, y_val (arrays): Training and validation datasets.
          - v_early (list): Early stopping information for each hyperparameter combination.
          - v_hist (list): Training history for each hyperparameter combination.
    """
    
    
    bestHyperparameters = {'dropout': -1, 'layers': -1, 'lr_scheduler':-1}
    bestMetricDev = np.inf
    
    
    lr_scheduler = hyperparameters["lr_scheduler"]
    layers = hyperparameters["layers"]
    dropout = hyperparameters["dropout"]
    
    for k in range(len(dropout)):
        for l in range(len(layers)):
            for m in range(len(lr_scheduler)):
                hyperparameters = {
                    'timeStep': hyperparameters["timeStep"],
                    'maskValue': hyperparameters["maskValue"],
                    'batch_size': hyperparameters["batch_size"],
                    'epochs': hyperparameters["epochs"],
                    'monitor':  hyperparameters["monitor"],
                    "mindelta": hyperparameters["mindelta"],
                    "patience": hyperparameters["patience"],
                    "kfold": hyperparameters["kfold"],
                    
                    "dropout": dropout[k],
                    "layers": layers[l],
                    "lr_scheduler": lr_scheduler[m],
                
                    'verbose': 0
                }
                v_metric_dev = []
                v_hist = []
                v_val_loss = []
                print("\t\tLearning rate:", lr_scheduler[m], ", dropout:", dropout[k], ", layers:", layers[l])

               
                all_patients_train = X_train.shape[0]
                kf = KFold(n_splits=hyperparameters["kfold"], shuffle=True, random_state=seed)
                kf.get_n_splits(all_patients_train)
                for train_index, val_index in kf.split(X_train):
                    X_train_cv = X_train[train_index]
                    X_val_cv = X_train[val_index]
                    y_train_cv = y_train[train_index]
                    y_val_cv = y_train[val_index]
                    
                    reset_keras()
                    model = GRUModel(hyperparameters)
                    hist, model = model.train(X_train_cv, y_train_cv, X_val_cv, y_val_cv)
                
                    v_hist.append(hist)
                    v_val_loss.append(np.min(hist.history["val_loss"]))
                    
                metric_dev = np.mean(v_val_loss)
                if metric_dev < bestMetricDev:
                    print("\t\t\tCambio the best", bestMetricDev, "por metric dev:", metric_dev)
                    bestMetricDev = metric_dev
                    bestHyperparameters['dropout'] = k
                    bestHyperparameters['layers'] = l
                    bestHyperparameters['lr_scheduler'] = m

    return bestHyperparameters, X_train_cv, X_val_cv, y_train_cv, y_val_cv

## Hyperparameters

In [3]:
# Hyperparameters
seeds = [143, 45, 67]

input_shape = 7
# Select the first 24h - 24 time steps
timeStep = 6
batch_size = 32
epochs = 1000

layer_list = [
    [input_shape, 5, 1],
    [input_shape, 3, 1],
]
dropout = [0.0, 0.15, 0.3, 0.45]
lr_scheduler = [0.1, 0.01, 0.001, 0.0001]


hyperparameters = {
    "timeStep": timeStep,
    "maskValue": 666,
    "batch_size": batch_size,
    "epochs": epochs,
    "monitor": "val_loss",
    "mindelta": 0,
    "patience": 50,
    "kfold": 5,
    "dropout": dropout,
    "lr_scheduler": lr_scheduler,
    "layers": layer_list,
    "verbose": 0,
}

## Model execution

In [4]:
metrics_data = []

loss_train = []
loss_dev = []
v_models = []
standard_scores_data = []
bestHyperparameters_bySplit = {}

folders = ["s1", "s2", "s3"]
init_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  

for i in range(0,3):

    X_train = np.load("../0_Data/splits/" + folders[i] + "/X_train_tensor.npy")
    y_train = np.load("../0_Data/splits/" + folders[i] + "/y_train_tensor.npy")
    y_train = y_train.reshape(y_train.shape[0], y_train.shape[1], 1)
    
    X_test = np.load("../0_Data/splits/" + folders[i] + "/X_test_tensor.npy")
    y_test = np.load("../0_Data/splits/" + folders[i] + "/y_test_tensor.npy")
    y_test = y_test.reshape(y_test.shape[0], y_test.shape[1], 1)

    #GridSearch of hyperparameters and print them   
    bestHyperparameters, X_train, X_val, y_train, y_val = myCVGrid(hyperparameters, seeds[i], X_train, y_train)
    
    bestHyperparameters_bySplit[str(i)] = bestHyperparameters
    print("\tlr_sch seleccionado:", lr_scheduler[bestHyperparameters["lr_scheduler"]])
    print("\tdropout seleccionado:", dropout[bestHyperparameters["dropout"]])
    print("\tlayers seleccionado:", layer_list[bestHyperparameters["layers"]])
    
    
    besthyperparameters = {
        'timeStep': hyperparameters["timeStep"],
        'maskValue': hyperparameters["maskValue"],
        'batch_size': hyperparameters["batch_size"],
        'epochs': hyperparameters["epochs"],
        'monitor':  hyperparameters["monitor"],
        "mindelta": hyperparameters["mindelta"],
        "patience": hyperparameters["patience"],                    
        "dropout": dropout[bestHyperparameters["dropout"]],
        "layers": layer_list[bestHyperparameters["layers"]],
        "lr_scheduler": lr_scheduler[bestHyperparameters["lr_scheduler"]],                    
        'kfold': hyperparameters["kfold"],
        'verbose': 0
    }
    
#--- TRY ON TEST -----------------------------------------------------------------------#

    #Reset keras
    reset_keras()
    model = GRUModel(besthyperparameters)
    hist, model = model.train(X_train, y_train, (X_val, y_val))

    y_pred = model.predict(x=X_test)
    y_pred = np.reshape(y_pred, (y_pred.size,))
        
# --- METRICS -----------------------------------------------------------------------#     
    accuracy_test = sklearn.metrics.accuracy_score(y_test2D, np.round(y_pred_final))
    tn, fp, fn, tp = confusion_matrix(y_test2D, np.round(y_pred_final)).ravel()
    roc = sklearn.metrics.roc_auc_score(y_test2D, y_pred_final)

    accuracy = accuracy_test
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn) 
    f1score =  (2 * precision * recall) / (precision + recall)
    
    metrics = {
        "S": i,  
        "TN": tn,
        "TP": tp,
        "FN": fn,
        "FP": fp,
        "ACC": accuracy,
        "SPEC": specificity,
        "PREC": precision,
        "RECALL": recall,
        "F1": f1score,
        "ROC": roc,
        "bestHyper": bestHyperparameters

    }
    metrics_data.append(metrics)
    break


df = pd.DataFrame(metrics_data)
df = df.sort_values(by='S')
df["Date_end"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")  
df["Date_in"] = init_date

		Learning rate: 0.1 , dropout: 0.0 , layers: [7, 5, 1]


AttributeError: 'tuple' object has no attribute 'as_list'